In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import re

from depsurf import RenameType, Version


def get_suffix(x):
    suffix = x.split(".", 1)[-1]
    return re.sub(r"\.\d+", "", suffix)


def count_rename(v: Version):
    all_names = {sym["name"]: sym["bind"] for sym in v.img.symtab.iter_funcs()}
    result = {t: 0 for t in RenameType}

    for func in all_names.keys():
        if "." not in func:
            continue
        suffix = get_suffix(func)
        if suffix == "localalias":
            continue
        result[RenameType.from_suffix(suffix)] += 1

    total = len(all_names)
    return {
        **{k: v / total for k, v in result.items()},
        "GCC": v.img.gcc_version,
    }

In [3]:
from depsurf import VersionGroup
from utils import save_pkl

groups = [VersionGroup.REGULAR, VersionGroup.ARCH]

data = {}
for group in groups:
    group_data = {}
    for version in group.versions:
        group_data[version] = count_rename(version)
    data[group] = group_data

save_pkl(data, "rename")

[        utils.py:160] INFO: NumExpr defaulting to 8 threads.
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.4.0-21-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.8.0-22-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.10.0-19-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.13.0-16-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.15.0-20-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.18.0-10-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/5.0.0-13-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Dow

In [4]:
import pandas as pd

from utils import load_pkl

data = load_pkl("rename")
pd.DataFrame(
    {
        (group, version): data[group][version]
        for group in data
        for version in data[group]
    }
).T

[          pkl.py:18 ] INFO: Loding rename from /Users/szhong/Downloads/bpf-study/output/rename.pkl


isra constprop      part      cold        ≥2     GCC
Regular 4.4.0-21-generic-amd64    0.014974  0.005104  0.013593       0.0  0.001667   5.3.1
        4.8.0-22-generic-amd64    0.014327  0.005785   0.01393       0.0  0.001564   6.2.0
        4.10.0-19-generic-amd64   0.014254  0.006036  0.013939       0.0   0.00143   6.3.0
        4.13.0-16-generic-amd64   0.014063  0.005788  0.016829       0.0  0.001604   7.2.0
        4.15.0-20-generic-amd64   0.014016  0.005754  0.016994       0.0  0.001455   7.3.0
        4.18.0-10-generic-amd64   0.014575  0.005409  0.015174  0.038709  0.003324   8.2.0
        5.0.0-13-generic-amd64    0.014549  0.005296  0.015199  0.038757  0.003186   8.3.0
        5.3.0-18-generic-amd64    0.014559  0.005139  0.014262  0.080952  0.006144   9.2.1
        5.4.0-26-generic-amd64    0.014583  0.005142  0.014359  0.060259  0.004562   9.3.0
        5.8.0-25-generic-amd64    0.010318  0.010969  0.016625  0.062061  0.006324  10.2.0
        5.11.0-16-generic-amd64   0.010071  0.010685    0.0161  0.061863  0.006353  10.3.0
        5.13.0-19-generic-amd64   0.008565  0.009926  0.012001  0.063574  0.005179  11.2.0
        5.15.0-25-generic-amd64   0.007788  0.009361  0.010797  0.123145  0.008368  11.2.0
        5.19.0-21-generic-amd64   0.008096   0.00996  0.007681  0.125993  0.007489  12.2.0
        6.2.0-20-generic-amd64    0.008389  0.010971  0.007374  0.085352  0.006269  12.2.0
        6.5.0-9-generic-amd64      0.01162  0.008104  0.007146  0.085798  0.006003  13.2.0
        6.8.0-31-generic-amd64    0.011493  0.007887  0.007158  0.088564  0.005875  13.2.0
Arch    5.4.0-26-generic-arm64     0.01868  0.005023  0.015164       0.0  0.001893   9.3.0
        5.4.0-26-generic-armhf         0.0  0.008785  0.015802       0.0  0.000079   9.3.0
        5.4.0-26-generic-ppc64el  0.013084  0.004398  0.014905       0.0  0.001466   9.3.0
        5.4.0-26-generic-s390x    0.018939  0.004713  0.015105       0.0   0.00213   9.3.0

In [6]:
from depsurf import RenameType, VersionGroup
from utils import load_pkl, format_yticks, get_legend_handles_labels, save_fig
import matplotlib.pyplot as plt
import numpy as np

COLORS = {
    RenameType.ISRA: "C0",
    RenameType.PART: "C1",
    RenameType.COLD: "C2",
    RenameType.CONSTPROP: "C3",
    RenameType.MULTIPLE: "C4",
}

GROUPS = {
    VersionGroup.REGULAR: "Linux Kernel and GCC Versions",
    VersionGroup.ARCH: "Arch for v5.4\n(GCC 9.3)",
}


data = load_pkl("rename")
fig, axs = plt.subplots(
    1,
    len(data),
    figsize=(5, 1.25),
    sharey=True,
    width_ratios=[len(v) for v in data.values()],
    gridspec_kw={"wspace": 0.1},
)


for group_idx, (ax, (group, group_data)) in enumerate(zip(axs, data.items())):
    for i, (version, version_data) in enumerate(group_data.items()):
        bottom = 0
        for k, v in version_data.items():
            if k not in COLORS:
                continue
            ax.bar(i, v, 0.7, label=k, bottom=bottom, color=COLORS[k])
            bottom += v

    labels = []
    for version, version_data in group_data.items():
        label = f"{group.version_to_str(version)}"
        if group == VersionGroup.REGULAR:
            gcc_version = version_data["GCC"]
            gcc_version = version.img.gcc_version.rsplit(".", 1)[0]
            label += f" / {gcc_version}"
        labels.append(label)
    ax.set_xticks(np.arange(len(labels)), labels)
    ax.set_xlabel(GROUPS[group], fontsize=8)
    ax.set_xticklabels(labels, fontsize=8, rotation=90)
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{x:.0%}"))

    if group == VersionGroup.REGULAR:
        ax.set_ylabel("Pct. of Functions")

handles, labels = get_legend_handles_labels(fig)
fig.legend(
    handles,
    labels,
    loc="upper center",
    ncol=5,
    bbox_to_anchor=(0.5, 1.1),
    fontsize=8,
    # handlelength=1.5,
    # handletextpad=0.5,
    # columnspacing=1,
)
save_fig(fig, "rename")

[          pkl.py:18 ] INFO: Loding rename from /Users/szhong/Downloads/bpf-study/output/rename.pkl
[          mpl.py:84 ] INFO: Saved figure to /Users/szhong/Downloads/bpf-study/paper/figs/rename.pdf


4.15 -> 4.18:
- `-freorder-blocks-and-partition` enabed by default in GCC 8

- https://gcc.gnu.org/gcc-8/changes.html

5.0 -> 5.3:
- "Hot/cold partitioning is now more precise and aggressive." since GCC 9

- https://gcc.gnu.org/gcc-9/changes.html

5.13 -> 5.15:

- Non-static functions renamed e.g., `ipv6_local_rxpmtu`

5.19 -> 6.2: 

- `-falign-functions=16` added since Linux 6.2

- https://github.com/torvalds/linux/commit/c27cd083cfb9d392f304657ed00fcde1136704e7

- https://github.com/torvalds/linux/commit/d49a0626216b95cd4bf696f6acf55f39a16ab0bb